**Cfile Maker: Defining (Mesh, Surface,...)**

In [1]:
# --- Imports ---
import pandas as pd
from pathlib import Path

# --- Inputs ---
file_path = 'position.csv'   # Update this with the actual file path
data = pd.read_csv(file_path)

# --- Mesh generation parameters ---
unit_scale = 10.0      # multiply cm values by this to get mm (10 for cm→mm)
mesh_dens = 6          # mesh density 
num_elems = 1513        # base number of elements per part 
num_nodes = 1652        # base number of nodes per part 
edge_size_l = 6         # adjust it to refine the tet meth (as it lager the mesh is finer)
edge_size_s = 3         # adjust it to refine the tet meth (as it lager the mesh is coarser)

# --- Function to create the .cfile---
def generate_cfile(data, output_file):
    with open(output_file, 'w') as f:
        # Write header
        f.write("$# LS-PrePost command file generated automatically\n")
        f.write("cemptymodel\n")
        f.write("genselect target node\n")
        f.write("genselect clear\n")
        
        # Loop through each row of the CSV file to create spheres
        for i, row in data.iterrows():
            # Convert cm → mm
            x = float(row["Points:0"]) * unit_scale
            y = float(row["Points:1"]) * unit_scale
            z = float(row["Points:2"]) * unit_scale
            radius = float(row["radius"]) * unit_scale
            # Part numbering starts at 1
            part_number = i + 1
            
            # Create sphere with
            f.write(f"meshing spheresolid create {x:.6f} {y:.6f} {z:.6f} {radius:.3f} {mesh_dens} 1 0 0 0 1 0\n")
            f.write("ac\n")
            # Accept mesh for the sphere with appropriate node ID ranges
            f.write(f"meshing spheresolid accept {part_number} {num_elems * part_number} {num_nodes * part_number} P{part_number}\n")
            f.write("ac\n")
        
        # Loop again to create the meshing commands for each part using the provided tetrahedral meshing commands
        for i, row in data.iterrows():
            part_number = i + 1  # Part number starts from 1
            radii = radius
            
            min_edge_size = radii / edge_size_l      # The minimum edge size for a tet mesh
            max_edge_size = radii / edge_size_s      # The maximum edge size for a tet mesh

            # Tetrahedral meshing commands using the provided code
            skin_command = f"tetmesh skin {part_number} 0 0 0\n"
            remesh_command = f"tetmesh remesh {min_edge_size} {max_edge_size} 1\n"
            mesh_command = "tetmesh mesh 0\n"
            accept_mesh_command = f"tetmesh accept {part_number + 10000} 0\n"
            xform_command = f"-M X{part_number}\n"

            # Write these commands to the file
            f.write(skin_command)
            f.write(remesh_command)
            f.write(mesh_command)
            f.write(accept_mesh_command)
            f.write(xform_command)

        # Additional segment and solid commands for each part
        for i in range(1, len(data) + 1):
            # Segment selection and set creation for each part
            f.write("genselect target part\n")
            f.write("setsegment\n")
            f.write("genselect target segment\n")
            f.write("genselect clear\n")
            f.write("genselect clear\n")
            f.write(f"genselect element add part {10000 + i}/0\n")
            f.write(f"setsegment createset {i} 1 0 0 0 0 \"Surf_P{i}\"\n")
            f.write("genselect clear\n")
            f.write("setsegment\n")
            f.write("genselect target segment\n")
            f.write("genselect clear\n")
            f.write("genselect clear\n")
            f.write("selectentity All\n")
            f.write("selectentity None\n")
        
        # Solid selection and set creation for each part
        for i in range(1, len(data) + 1):
            f.write("setsolid\n")
            f.write("genselect target solid\n")
            f.write("genselect clear\n")
            f.write("genselect solid add part {}/0\n".format(10000 + i))
            f.write("setsolid createset {} 0 \"s{}\"\n".format(i, i))
            f.write("genselect clear\n")



    print(f"Cfile generated successfully: {output_file}")

# Example usage: Call the function with your CSV data and desired output file name
generate_cfile(data, "Mesh_file.cfile")


Cfile generated successfully: Mesh_file.cfile


**Material Assignment**

In [2]:
def material_assigment_content(total_particles, output_file_path):
    content = ""
    for i in range(1, total_particles + 1):
        pid = i
        secid = i
        mid = 1
        content += f"*PART\n"
        content += f"*SECTION_SOLID_TITLE\n"
        content += f"P{i}-section\n"
        content += f"$#   secid    elform       aet    unused    unused    unused    cohoff   gaskeit\n"
        content += f"{secid:10d}{1:10d}{0:10d}{0:10d}{0:10d}{0:10d}{0.0:10.1f}{0.0:10.1f}\n"
        content += f"*PART\n"
        content += f"$#                                                                         title\n"
        content += f"New part from tetrahedron mesher\n"
        content += f"$#     pid     secid       mid     eosid      hgid      grav    adpopt      tmid\n"
        content += f"{10000 + pid:10d}{secid:10d}{mid:10d}{0:10d}{0:10d}{0:10d}{0:10d}{0:10d}\n"
    
    # Write to the output file
    with open(output_file_path, "w") as file:
        file.write(content)

# Define the number of particles and output file path
total_particles = 403                   # Write your total number of particle
output_file_path = "material.k"         # Update this with the actual file path

# Generate the corrected .k file
material_assigment_content(total_particles, output_file_path)

print(f"The material assigment file has been generated in '{output_file_path}'.")


The material assigment file has been generated in 'material.k'.
